In [1]:
! ls -alh /var/model

total 432K
drwxr--r-- 35 1001 1001 4.0K Jan 11 20:33 .
drwxr-xr-x  1 root root 4.0K Jan 17 15:12 ..
-rw-r--r--  1 1001 1001 8.1K Jan  4 22:17 .DS_Store
-rw-r--r--  1 1001 1001 4.0K Jan  4 01:11 ._.DS_Store
drwxr--r--  3 1001 1001 4.0K Aug 22 13:57 .idea
drwxr--r-- 16 1001 1001 4.0K Aug 15 19:52 Auto-GPT
drwxrwxr-x  6 1001 1001 4.0K Nov 19 02:04 CodeLlama-13b-Instruct-hf
drwxrwxr-x  7 1001 1001 4.0K Dec 19 14:10 CodeLlama-7b-Instruct-hf
-rwxr--r--  1 1001 1001 3.3K Feb 28  2023 Dockerfile
drwxrwxr-x 15 1001 1001 4.0K Nov  8 18:06 OpenLLM
drwxrwxr-x  9 1001 1001 4.0K Jan 12 20:09 Phind-CodeLlama-34B-v2
drwxrwxr-x 13 1001 1001 4.0K Dec 13 00:08 TTS
drwxr--r--  6 1001 1001 4.0K Feb 28  2023 ampe
drwxr--r--  5 1001 1001 4.0K Jul  5  2023 cache
-rwxr--r--  1 1001 1001 254K Aug  1 01:17 chat_history.txt
drwxr--r--  5 1001 1001 4.0K Apr 28  2023 coursework
drwxrwxr-x  8 1001 1001 4.0K Nov  3 15:33 cursor-flask-chatbot
drwxr--r--  6 1001 1001 4.0K Sep  3 23:49 cursor-flask-js-websocket
drwxr-xr

In [2]:
%load_ext autoreload

import os

import pandas as pd

print( os.getcwd() )
# change working directory
os.chdir( "/var/model/genie-in-the-box/src" )
print( os.getcwd() )
import lib.utils.util as du

/
/var/model/genie-in-the-box/src


In [3]:
# associate commands with files containing examples
browser_commands = {
    "search new tab": "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt",
    "search current tab": "/src/ephemera/prompts/data/synthetic-data-search-in-current-tab.txt",
    "search google new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-new-tab.txt",
    "search google current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-current-tab.txt",
    "search google scholar new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-new-tab.txt",
    "search google scholar current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-current-tab.txt",
}

In [4]:
prefix = "/var/model/genie-in-the-box"
# prefix = du.get_project_root()

for command in browser_commands.keys():
    print( os.path.exists( prefix + browser_commands[ command ] ) )

True
True
True
True
True
True


In [8]:
raw_lines = du.get_file_as_list( prefix + "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt", clean=True, randomize=False )[ 0:100 ]
raw_lines[ 0:10 ]

['SEARCH_TERMS new tab search',
 'New tab, search SEARCH_TERMS',
 'Open new tab, search for SEARCH_TERMS',
 'Pull up SEARCH_TERMS in another tab',
 'Find SEARCH_TERMS in a new tab',
 'Hunt for SEARCH_TERMS, new tab',
 'Display SEARCH_TERMS results in new tab',
 'Do a search for SEARCH_TERMS and show results in another tab',
 'Execute SEARCH_TERMS search in a new tab',
 'Seek SEARCH_TERMS in a fresh tab']

In [9]:
def get_search_terms( requested_length ):
    
    # Load search terms file
    search_terms = du.get_file_as_list( prefix + "/src/ephemera/prompts/data/search-terms.txt", lower_case=False, clean=True, randomize=True )
    
    # If we don't have enough search terms, append copies of the search term list until we do
    while requested_length > len( search_terms ):
        search_terms += search_terms
        
    # Truncate the search terms list to equal the requested len
    search_terms = search_terms[ :requested_length ]
    
    return search_terms

In [10]:
search_terms = get_search_terms( len( raw_lines ) )
search_terms[ 0:10]

['Indentation Error',
 'Using Pandas in web development',
 'Exporting data from Pandas to Excel',
 'ResourceWarning',
 'what is the meaning of success?',
 'AI in space exploration',
 'Data compression in Pandas',
 'Using apply and map in Pandas',
 'popular movie genres',
 'Pandas DataFrame reshaping techniques']

In [11]:
command_choices = "'" + "', '".join( browser_commands.keys() ) + "' and 'none'"
command_choices

"'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'"

In [98]:
gpt_instruction = f"""INSTRUCTIONS:
Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command as INPUT as well as a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

RESPONSE FORMAT: MUST be returned wrapped in simple, well-formed XML
<response>
    <browser-command></browser-command>
    <args></args>
</response>
"""

# gpt_instruction = f"""
# INSTRUCTIONS:
# Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.
# 
# You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.
# 
# Requirement: You MUST NOT use python code to answer this question.
# Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
# Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.
# 
# INPUT FORMAT: You will receive a raw human voice command transcription tagged using simple XML
# <human>
#     <voice-command></voice-command>
# </human>
# 
# RESPONSE FORMAT: MUST be returned wrapped in simple, well-formed XML
# <response>
#     <browser-command></browser-command>
#     <args></args>
# </response>
# """

# gpt_input_template = """
# INPUT:
# <human>
#     <voice-command>{voice_command}</voice-command>
# </human>
# 
# YOUR RESPONSE:
# """


output_template = """
<response>
    <browser-command>{browser_command}</browser-command>
    <args>{args}</args>
</response>"""

In [99]:
for line in gpt_instruction.split( "\n" ):
    print( line )

INSTRUCTIONS:
Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command as INPUT as well as a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

RESPONSE FORMAT: MUST be returned wrapped in simple, well-formed XML
<response>
    <browser-command></browser-command>
    <args></args>
</response>


In [100]:

instruction_template = """Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command."""

input_template = """
Below is the raw human voice command transcription formatted using simple XML: 
{human_says}

The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
{response_format}"""

human_says_template = """
<human>
    <voice-command>{voice_command}</voice-command>
</human>"""

response_format = """
<response>
    <browser-command></browser-command>
    <args></args>
</response>

Requirement: The first word of your response MUST be `<response>`"""

output_template = """
<response>
    <browser-command>{browser_command}</browser-command>
    <args>{args}</args>
</response>"""

In [23]:
for line in instruction_template.split( "\n" ):
    print( line )

Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.


In [16]:
instruction = instruction_template.format( command_choices=command_choices )
instruction

"Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.\n\nYou will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.\n\nRequirement: You MUST NOT use python code to answer this question.\nRequirement: You MUST use your linguistic knowledge and intuition to answer this question.\nHint: Anything that isn't a part of the command itself should be treated as arguments related to the command."

In [17]:
def prompt_instruction_format( instruction, input ):

    return f"""### Instruction:
Use the Task and Input given below to write a Response that can solve the following Task.

### Task:
{instruction}

### Input:{input}
### Response:
"""

In [ ]:
messages = [
    { "role": "system", "content": gpt_instruction },
    { "role": "user", "content": voice_command },
    # { "role": "user", "content": gpt_input_template.format( voice_command=voice_command ) },
    { "role": "assistant", "content": output_template.format( browser_command=browser_command, args=terms ) }
]
messages

In [123]:
instructions = []
inputs       = []
outputs      = []
prompts      = []
# gpt_prompts  = []
# gpt_inputs   = []
gpt_messages = []

for browser_command in browser_commands.keys():
    
    du.print_banner( browser_command, prepend_nl=True, end="\n" )
    
    raw_lines = du.get_file_as_list( prefix + browser_commands[ browser_command ], clean=True )
    for line in raw_lines[ 0:100 ]: #[ 0:2 ]:#
        
        # get newly randomized search terms on every iteration
        for terms in get_search_terms( len( raw_lines ) ):#[ 0:10 ]: #[ 0:2]:#:
            
            voice_command = line.replace( "SEARCH_TERMS", terms )
            instruction   = instruction_template.format( command_choices=command_choices )
            human_says    = human_says_template.format( voice_command=voice_command )
            input         = input_template.format( human_says=human_says, response_format=response_format )
            # gpt_input     = gpt_input_template.format( voice_command=voice_command )
            output        = output_template.format( browser_command=browser_command, args=terms )
            prompt        = prompt_instruction_format( instruction, input )
    
            instructions.append( instruction )
            inputs.append( input )
            outputs.append( output )
            prompts.append( prompt )
            # gpt_inputs.append( gpt_input )
            # gpt_prompts.append( gpt_instruction )
            gpt_messages.append( { 
                "messages": [
                    { "role": "system", "content": gpt_instruction },
                    { "role": "user", "content": voice_command },
                    { "role": "assistant", "content": output_template.format( browser_command=browser_command, args=terms ) }
                ] 
            } )
    
            print( ".", end="" )
        
    print()


------------------------------------------------------------------------------------------------------------------------
- search new tab
------------------------------------------------------------------------------------------------------------------------
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [102]:
gpt_messages[ 0 ]

[{'role': 'system',
  'content': "INSTRUCTIONS:\nYour job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.\n\nYou will be given a human voice command as INPUT as well as a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.\n\nRESPONSE FORMAT: MUST be returned wrapped in simple, well-formed XML\n<response>\n    <browser-command></browser-command>\n    <args></args>\n</response>\n"},
 {'role': 'user', 'content': 'what is the stock market? new tab search'},
 {'role': 'assistant',
  'content': '\n<response>\n    <browser-command>search new tab</browser-command>\n    <args>what is the stock market?</args>\n</response>'}]

In [103]:
for line in prompts[ 0 ].split( "\n" ):
    print( line )

### Instruction:
Use the Task and Input given below to write a Response that can solve the following Task.

### Task:
Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

### Input:
Below is the raw human voice command transcription formatted using simple XML: 

<human>
    <v

In [38]:
idx = 0

for line in instructions[ idx ].split( "\n" ):
    print( line )
    
for line in inputs[ idx ].split( "\n" ):
    print( line )
    
for line in outputs[ idx ].split( "\n" ):
    print( line )

Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

Below is the raw human voice command transcription formatted using simple XML: 

<human>
    <voice-command>best hiking trails near me new tab search</voice-command>
</human>

The standardized command that you translate MUST

In [39]:
import random

def inject_space( line ):
    if random.random() < 0.5:
        index = random.randint( 0, len( line ) - 1 )
        return line[ :index ] + ' ' + line[ index: ]
    else:
        return line

In [104]:
qna_df = pd.DataFrame( { "instruction": instructions, "input": inputs, "output": outputs, "prompt": prompts, "gpt_message": gpt_messages } )

# Perturb the answer to create a faked response  
qna_df[ "response" ] = qna_df[ "output" ].apply( lambda cell: inject_space( cell ) )
qna_df.gpt_messages.head()

0    [{'role': 'system', 'content': 'INSTRUCTIONS:
...
1    [{'role': 'system', 'content': 'INSTRUCTIONS:
...
2    [{'role': 'system', 'content': 'INSTRUCTIONS:
...
3    [{'role': 'system', 'content': 'INSTRUCTIONS:
...
4    [{'role': 'system', 'content': 'INSTRUCTIONS:
...
Name: gpt_messages, dtype: object

## Validate the structure of the xml response

In [49]:
!pip install xmlschema

In [50]:
from xmlschema import XMLSchema

xml_str_1 = """
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""
xml_str_2 = """
<response>
  <browser_command></browser_command>
  <args></args>
</response>
"""
xml_str_3 = """
Sure here's your command!
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""

xml_str_4 = """
```xml
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""

xsd_string = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="response">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="browser-command" type="xs:string"/>
        <xs:element name="args" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

def is_valid_xml( xml_str, schema ):
    
    try:
        schema.is_valid( xml_str )
        return True
    except Exception as e:
        # print( "Invalid XML: " + str( e ) )
        return False

schema   = XMLSchema( xsd_string )

print( is_valid_xml( xml_str_1, schema ) )  # Output: ???
print( is_valid_xml( xml_str_2, schema ) )  # Output: ???
print( is_valid_xml( xml_str_3, schema ) )  # Output: False
print( is_valid_xml( xml_str_4, schema ) )  # Output: False

True
True
False
False


In [51]:
qna_df[ "response_xml_is_valid" ] = qna_df[ "response" ].apply( lambda cell: is_valid_xml( cell, schema ) )
qna_df.head()

,instruction,input,output,prompt,gpt_messages,response,response_xml_is_valid
0,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True
1,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True
2,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,False
3,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True
4,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new...,True


In [52]:
qna_df.response_xml_is_valid.mean()

0.7940583333333333

In [53]:
def contains_valid_xml_tag( xml_str, tag_name ):
    
    try:
        return "<" + tag_name + ">" in xml_str and "</" + tag_name + ">" in xml_str
    except Exception as e:
        return False
    
contains_valid_xml_tag( "```xml<response><browser-command></browser-command><args></args></response>", "browser-command")

True

## Validate the structure of the xml response

In [54]:
qna_df[ "contains_response" ]        = qna_df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "response" ) )
qna_df[ "contains_browser_command" ] = qna_df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "browser-command" ) )
qna_df[ "contains_args" ]            = qna_df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "args" ) )

In [55]:
print( qna_df[ "contains_response" ].mean() )
print( qna_df[ "contains_browser_command" ].mean() )
print( qna_df[ "contains_args" ].mean() )

0.9306083333333334
0.8822166666666666
0.9600916666666667


## Validate the response by removing all white space and comparing to the original answer

In [56]:
import re

xml_str = """
<response>
  <browser-command>Whos your favorite browser?</browser-command>
  <args>Bar browser</args>
</response>
"""

# Remove white space outside XML tags
xml_str = re.sub(r'>\s+<', '><', xml_str.strip())

print(xml_str)

<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>


In [57]:
def is_response_exact( response, answer ):
    
    # Remove white space outside XML tags
    response = re.sub( r'>\s+<', '><', response.strip() )
    
    # Remove white space outside XML tags
    answer = re.sub( r'>\s+<', '><', answer.strip() )
    
    return response == answer

is_response_exact( "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>", "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>" )

True

In [58]:
def tag_values_are_equal( response, answer, tag_name="browser-command" ):
    
    command_response = dux.get_value_by_xml_tag_name( response, tag_name, default_value="broken" )
    command_answer   = dux.get_value_by_xml_tag_name(   answer, tag_name, default_value="broken" )
    
    return command_response != "broken" and command_answer != "broken" and command_response == command_answer

response = "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>"
answer   = "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>"

print( tag_values_are_equal( response, answer, tag_name="browser-command" ) )
print( tag_values_are_equal( response, answer, tag_name="args" ) )

NameError: name 'dux' is not defined

In [59]:
import lib.utils.util_xml as dux

def contains_correct_response_values( response, answer ):
    
    """Check to see if the most common formatting error (```xml) is hiding a correct response"""
    response = "<response>" + dux.get_value_by_xml_tag_name( response, "response", default_value="broken" ) + "</response>"
    response = dux.get_xml_tag_and_value_by_name( response, "response", default_value="broken" )
    if response == "broken":
        return False
    
    # Remove white space outside XML tags
    response = re.sub( r'>\s+<', '><', response.strip() )
    
    # Remove white space outside XML tags
    answer = re.sub( r'>\s+<', '><', answer.strip() )
    
    return response == answer

contains_correct_response_values( "```xml<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>", "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>" )

True

In [60]:
qna_df[ "response_is_correct" ] = qna_df.apply( lambda row: is_response_exact( row[ "response" ], row[ "output" ] ), axis=1 )

In [61]:
qna_df.response_is_correct.mean()

0.5582

In [62]:
qna_df[ ( qna_df.response_xml_is_valid == True ) & ( qna_df.response_is_correct == False ) ].head( 10 )

,instruction,input,output,prompt,gpt_messages,response,response_xml_is_valid,contains_response,contains_browser_command,contains_args,response_is_correct
10,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True,True,True,True,False
17,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True,True,True,True,False
22,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True,True,True,True,False
26,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True,True,True,True,False
28,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True,True,True,True,False
31,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True,True,True,True,False
32,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True,True,True,True,False
34,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new...,True,True,True,True,False
36,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response >\n <browser-command>search new...,True,False,True,True,False
44,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...,"[{'role': 'system', 'content': ' INSTRUCTIONS:...",\n<response>\n <browser-command>search new ...,True,True,True,True,False


# What's the baseline behavior of the Vanilla model?

In [12]:
# !pip install openai
# !pip install tiktoken

In [83]:
%autoreload
from huggingface_hub import InferenceClient
from lib.utils.util_stopwatch import Stopwatch
from lib.agents.agent import Agent

def query_llm_phind( prompt, model=Agent.PHIND_34B_v2, max_new_tokens=1024, temperature=0.25, top_k=10, top_p=0.9, debug=False, verbose=False, silent=True ):
    
    timer = Stopwatch( msg=f"Asking LLM [{model}]...".format( model ), silent=silent )
    
    # Get the TGI server URL for this context
    # default_url    = self.config_mgr.get( "tgi_server_url_phind", default=None )
    # tgi_server_url = du.get_tgi_server_url_for_this_context( default_url=default_url )
    
    client         = InferenceClient( "http://172.17.0.5:3000" )
    token_list     = [ ]
    ellipsis_count = 0
    
    if debug:
        for line in prompt.split( "\n" ):
            print( line )
    
    for token in client.text_generation(
        prompt, max_new_tokens=max_new_tokens, stream=True, temperature=temperature, top_k=top_k, top_p=top_p, stop_sequences=[ "</response>" ]
    ):
        if debug:
            print( token, end="" )
        else:
            if not silent: print( ".", end="" )
            ellipsis_count += 1
            if ellipsis_count == 120:
                ellipsis_count = 0
                print()
            
        token_list.append( token )
        
    response = "".join( token_list ).strip()
    
    timer.print( msg="Done!", use_millis=True, prepend_nl=True, end="\n" )
    tokens_per_second = len( token_list ) / ( timer.get_delta_ms() / 1000.0 )
    print( f"Tokens per second [{round( tokens_per_second, 1 )}]" )
            
    if debug:
        print( f"Token list length [{len( token_list )}]" )
        if verbose:
            for line in response.split( "\n" ):
                print( line )
    
    
    return response

In [137]:
import openai
from lib.utils.util_stopwatch import Stopwatch

def query_llm_openai( messages, model="gpt-3.5-turbo-1106", debug=False, verbose=False ):
    
    openai.api_key = du.get_api_key( "openai", project_root="/var/model/genie-in-the-box" )
    
    timer = Stopwatch( msg=f"Asking LLM [{model}]...".format( model ) )
    
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens=256,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    
    timer.print( "Done!", use_millis=True )
    if debug and verbose:
        # print( json.dumps( response.to_dict(), indent=4 ) )
        print( response )
    
    return response.choices[ 0 ].message.content.strip()

In [53]:
!pip show openai

Name: openai
Version: 1.5.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [69]:
call_counter = 0
# rows         = df.shape[ 0 ]

def get_response_to_question( prompt, rows ):
    
    global call_counter
    
    call_counter += 1
    print( f"On call [{call_counter:03d}] out of [{rows}] = [{round( call_counter / rows * 100.0, 1 )}%]... ", end="" )
    
    return query_llm_phind( prompt )

In [143]:
call_counter = 0

def get_gpt_response_to_question( cell, rows, model="gpt-3.5-turbo-1106" ):
    
    global call_counter    
    call_counter += 1
    
    print( f"On call [{call_counter:03d}] out of [{rows}] = [{round( call_counter / rows * 100.0, 1 )}%]... ", end="" )
    
    return query_llm_openai( cell[ "messages" ], model=model )

In [71]:
qna_df.shape

(120000, 11)

In [144]:
def validate_responses( df ):
    
    rows = df.shape[ 0 ]
    global call_counter
    call_counter = 0
    
    # df[ "response" ]                    = df[ "prompt" ].apply( lambda cell: get_response_to_question( cell, rows ) )
    df[ "response" ]                    = df[ "gpt_message" ].apply( lambda cell: get_gpt_response_to_question( cell, rows ) )
    
    # Validate the structure and content of the xml response
    df[ "response_xml_is_valid" ]       = df[ "response" ].apply( lambda cell: is_valid_xml( cell, schema ) )
    df[ "contains_response" ]           = df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "response" ) )
    df[ "contains_browser_command" ]    = df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "browser-command" ) )
    df[ "contains_args" ]               = df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "args" ) )
    df[ "response_is_exact" ]           = df.apply( lambda row: is_response_exact( row[ "response" ], row[ "output" ] ), axis=1 )
    df[ "response_has_correct_values" ] = df.apply( lambda row: contains_correct_response_values( row[ "response" ], row[ "output" ] ), axis=1 )
    
    return df


In [96]:
def print_stats( df ):

    du.print_banner( "Stats" )
    print( f"               Is valid xml {df.response_xml_is_valid.mean() * 100:.1f}%" )
    print( f"          Contains response {df.contains_response.mean() * 100:.1f}%" )
    print( f"   Contains browser command {df.contains_browser_command.mean() * 100:.1f}%" )
    print( f"              Contains args {df.contains_args.mean() * 100:.1f}%" )
    print( f"          Response is exact {df.response_is_exact.mean() * 100:.1f}%" )
    print( f"Response has correct values {df.response_has_correct_values.mean() * 100:.1f}%" )
    

In [142]:
gpt_messages[ 0 ][ "messages" ]

[{'role': 'system',
  'content': "INSTRUCTIONS:\nYour job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.\n\nYou will be given a human voice command as INPUT as well as a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.\n\nRESPONSE FORMAT: MUST be returned wrapped in simple, well-formed XML\n<response>\n    <browser-command></browser-command>\n    <args></args>\n</response>\n"},
 {'role': 'user',
  'content': 'Big Data analysis techniques: What techniques are essential for effective big data analysis? new tab search'},
 {'role': 'assistant',
  'content': '\n<response>\n    <browser-command>search new tab</browser-command>\n    <args>Big Da

In [125]:
qna_df = pd.DataFrame( { "instruction": instructions, "input": inputs, "output": outputs, "prompt": prompts, "gpt_message": gpt_messages } )
qna_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   instruction   120000 non-null  object
 1   input         120000 non-null  object
 2   output        120000 non-null  object
 3   prompt        120000 non-null  object
 4   gpt_messages  120000 non-null  object
dtypes: object(5)
memory usage: 4.6+ MB


In [126]:
from sklearn.model_selection import train_test_split

sampled_df = qna_df[ [ "instruction", "input", "output", "prompt", "gpt_message" ] ].sample( 1000, random_state=42 ).copy()

# Split the dataframe into train and (test+validate)
train_df, test_validate_df = train_test_split( sampled_df, test_size=0.2, random_state=42 )

# Then split (test+validate) into test and validate
test_df, validate_df = train_test_split( test_validate_df, test_size=0.5, random_state=42 )


In [127]:
print( train_df.shape )
print( test_df.shape )
print( validate_df.shape )

(800, 5)
(100, 5)
(100, 5)


In [145]:
validate_df = validate_responses( validate_df )

print_stats( validate_df )

# First run before attempting to force valid XML
# ------------------------------------------------------------------------------------------------------------------------
# - Stats: Phind
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 0.0%
#           Contains response 98.0%
#    Contains browser command 98.0%
#               Contains args 98.0%
#           Response is exact 0.0%
# Response has correct values 43.0%

# after attempting to force the first token output: <response>
# ------------------------------------------------------------------------------------------------------------------------
# - Stats: Phind
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 0.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 0.0%
# Response has correct values 39.0%

# ------------------------------------------------------------------------------------------------------------------------
# - Stats: GPT 3.5, before training
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact  88.0%
# Response has correct values  88.0%
# 
# ------------------------------------------------------------------------------------------------------------------------
# - Stats: GPT 3.5, after OVER training
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 57.0%
# Response has correct values 57.0%

On call [001] out of [100] = [1.0%]... Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]...
Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]... Done! in 743 ms

On call [002] out of [100] = [2.0%]... Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]...
Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]... Done! in 830 ms

On call [003] out of [100] = [3.0%]... Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]...
Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]... Done! in 569 ms

On call [004] out of [100] = [4.0%]... Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]...
Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]... Done! in 640 ms

On call [005] out of [100] = [5.0%]... Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]...
Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]... Done! in 852 ms

On call [006] out of [100] = [6.0%]... Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]...
Asking LLM [ft:gpt-3.5-turbo-1106:deepily::8hmLrC7G]... Done! in 

In [147]:
validate_df.response.head()

65397     <response>\n    <browser-command>search google...
72078     <response>\n    <browser-command>search google...
59206     <response>\n    <browser-command>search google...
48642     <response>\n    <browser-command>search google...
114768    <response>\n    <browser-command>search google...
Name: response, dtype: object

In [109]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 32114 to 10744
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   instruction   80 non-null     object
 1   input         80 non-null     object
 2   output        80 non-null     object
 3   prompt        80 non-null     object
 4   gpt_messages  80 non-null     object
dtypes: object(5)
memory usage: 3.8+ KB


In [128]:
sampled_df = qna_df[ [ "instruction", "input", "output", "prompt", "gpt_message" ] ].sample( 1000, random_state=42 ).copy()

# Split the dataframe into 80% train and (test+validate)
train_df, test_validate_df = train_test_split( sampled_df, test_size=0.2, random_state=42 )

# Then split (test+validate) into 10% test and 10% validate
test_df, validate_df = train_test_split( test_validate_df, test_size=0.5, random_state=42 )

In [115]:
project_root = "/var/model/genie-in-the-box"
path = du.get_project_root() + "/src/ephemera/prompts/data/search-xml-train.jsonl"
train_df.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )

path = du.get_project_root() + "/src/ephemera/prompts/data/search-xml-test.jsonl"
test_df.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )

path = du.get_project_root() + "/src/ephemera/prompts/data/search-xml-validate.jsonl"
validate_df.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )


OSError: Cannot save file into a non-existent directory: '/var/genie-in-the-box/src/ephemera/prompts/data'

In [131]:
path = project_root + "/src/ephemera/prompts/data/search-xml-train-gpt.jsonl"
train_df.gpt_messages.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )

path = project_root + "/src/ephemera/prompts/data/search-xml-test-gpt.jsonl"
test_df.gpt_messages.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )


In [132]:
train_df.gpt_messages.head()

61754     {'messages': [{'role': 'system', 'content': 'I...
108836    {'messages': [{'role': 'system', 'content': 'I...
112152    {'messages': [{'role': 'system', 'content': 'I...
75196     {'messages': [{'role': 'system', 'content': 'I...
7770      {'messages': [{'role': 'system', 'content': 'I...
Name: gpt_messages, dtype: object

In [120]:
! ls -alh /var/model/genie-in-the-box/src/ephemera/prompts/data

total 32M
drwxr--r-- 4 1001 1001 4.0K Jan 16 18:26 .
drwxr--r-- 4 1001 1001 4.0K Jan 11 04:58 ..
-rwxr--r-- 1 1001 1001 6.1K Jun 20  2023 .DS_Store
-rwxr--r-- 1 1001 1001 4.0K Jun 20  2023 ._.DS_Store
-rw-rw-r-- 1 1001 1001    0 Oct 13 18:48 __init__.py
drwxrwxr-x 2 1001 1001 4.0K Oct 13 18:48 fine-tuning-results
drwxrwxr-x 2 1001 1001 4.0K Oct 13 18:48 jsonl
-rw-rw-r-- 1 1001 1001 1.6K Oct 13 18:48 munger.py
-rw-rw-r-- 1 1001 1001  36K Dec 18 19:16 search-terms.txt
-rw-rw-rw- 1 root root 583K Jan 16 18:31 search-xml-train-gpt.jsonl
-rw-rw-r-- 1 1001 1001 7.0K Oct 13 18:48 synthetic-data-load-url-in-current-tab.txt
-rw-rw-r-- 1 1001 1001 7.3K Oct 13 18:48 synthetic-data-load-url-new-tab.txt
-rw-rw-r-- 1 1001 1001  13K Oct 13 18:48 synthetic-data-none-of-the-above.txt
-rw-rw-r-- 1 1001 1001 8.5K Dec 17 00:42 synthetic-data-search-google-in-current-tab.txt
-rw-rw-r-- 1 1001 1001  11K Oct 13 18:48 synthetic-data-search-google-in-new-tab.txt
-rw-rw-r-- 1 1001 1001 8.5K Oct 13 18:48 synthet

In [133]:
import torch

getattr(torch, "float16")

torch.float16